In [1]:
# For data 
import pandas as pd
# For datetime manipulation.
import datetime as dt
from datetime import time
import time
# For Nan.
import numpy as np
import numpy,math
# For numerical analysis
from scipy.stats import norm
from scipy.optimize import brentq

In [3]:
# Newton Raphson using black scholes -- Functions only
def compute_d1(strike, spot, time_to_expiry, rate, vol):
     return (math.log(spot / strike) + (rate + 0.5 * vol**2) * time_to_expiry) / (vol * math.sqrt(time_to_expiry))
def black_scholes_call_price(strike, spot, time_to_expiry, rate, vol):  # Pricing call option using Black Scholes 
    d1 = compute_d1(strike, spot, time_to_expiry, rate, vol)
    d2 = d1 - vol * math.sqrt(time_to_expiry)
    return spot * norm.cdf(d1) - strike * math.exp(-rate * time_to_expiry) * norm.cdf(d2) # S N(d1) - Ke^{-rT}N(d2)  
def vega(strike, spot, time_to_expiry, rate, vol):  #Derivative of Call Option price w.r.t. volatility.
    d1 = compute_d1(strike, spot, time_to_expiry, rate, vol)
    return spot * norm.pdf(d1) * math.sqrt(time_to_expiry)   
def newton_raphson_black_scholes(seed, strike, spot, time_to_expiry, rate, call_value): 
    vol = float(seed)  
    start = time.time()
    for i in range(100):  
        price = black_scholes_call_price(strike, spot, time_to_expiry, rate, vol)
        diff = price - call_value
        if abs(diff) < 1e-5:
            end = time.time()
            return [round(vol, 4) if vol > 0 else np.nan, end - start, i+1]
        v = vega(strike, spot, time_to_expiry, rate, vol) 
        if abs(v) < 1e-8: # If vega drops a lot then the root shoots up  
            end = time.time()
            return [np.nan, end - start, i+1]
        vol = vol - diff / v
    end = time.time()
    return [np.nan, end - start, 100]

In [4]:
def implied_vol_brentq(call_value, strike, spot, time_to_expiry, rate):
    def objective(vol):
        return black_scholes_call_price(strike, spot, time_to_expiry, rate, vol) - call_value

    start = time.time()
    try:
        iv, result = brentq(objective, 1e-4, 5.0, xtol=1e-5, full_output=True)
        end = time.time()
        return [iv, end - start, result.iterations]
    except ValueError:
        end = time.time()
        return [np.nan, end - start, 0]  # failed to converge

In [5]:
# 1 file analysis --> 
LOGFILE='Option Expiry 14-Aug-2025.csv'
data=pd.read_csv(LOGFILE);
data["timestamp"]=pd.to_datetime(data["timestamp"])
data["expiry"]=pd.to_datetime(data["expiry"]);
data=data[(data["type"]=="CALL") & (data["ltp"]>0)]

In [6]:
data["strike"]=(data["strike"]).astype(float)
# Without this getting a typeerror at the moment.
rate=0.1
data["Log_moneyness"]= np.log(data["spot price"]/data["strike"]) + rate*data["time_to_maturity"] 
# Code provided by CHATGPT.
def classify_log_moneyness(df, log_moneyness_col='Log_moneyness', option_type_col='type'):
    # Copy log moneyness so we don't modify original
    lm = df[log_moneyness_col].copy()
    
    # Convert log moneyness back to ratio
    ratio = np.exp(lm)

    # Classification
    conditions = [
        ratio > 1.05,
        (ratio > 1.01) & (ratio <= 1.05),
        (ratio >= 0.99) & (ratio <= 1.01),
        (ratio >= 0.95) & (ratio < 0.99),
        ratio < 0.95
    ]
    class_choices = ["Deep ITM", "Slightly ITM", "ATM", "Slightly OTM", "Deep OTM"]

    return np.select(conditions, class_choices, default="Unknown")
data['classification'] = classify_log_moneyness(data)

In [7]:
# Apply to DataFrame row-wise
data[["Implied Volatility NR","Time Taken NR", "Iterations NR"]] = data.apply(lambda row: newton_raphson_black_scholes(1,row["strike"],row["spot price"],row["time_to_maturity"],rate,row["ltp"]), axis=1, result_type="expand")
data[["Implied Volatility BQ","Time Taken BQ", "Iterations BQ"]] = data.apply(lambda row: implied_vol_brentq(row["ltp"],row["strike"],row["spot price"],row["time_to_maturity"],rate), axis=1,result_type="expand")
data.to_csv("Saved Computations for Options Expiry 14-Aug-2025.csv",index=False);

In [2]:
# Dataframe for Benchmark purposes Just Nan entries and Compute time: 
cols=['NAN ENTRIES', 'COMPUTE_TIME', 'MEAN ITERATIONS'];
benchmark=pd.DataFrame(columns=cols);
benchmark.index.name="Log Moneyness Classification Type"  

In [ ]:
# I filter data with different conditions and save benchmarks accordingly. String will be used to denote the conditions imposed
def save_benchmark(data1,string): 
    new_benchmark=pd.DataFrame(columns=benchmark.columns);
    new_benchmark.loc[string+" NR ",:]=[(data1["Implied Volatility NR"].isna()).sum(),(data1["Time Taken NR"]).sum(),(data1["Iterations NR"]).mean()]
    new_benchmark.loc[string+" BQ ",:]=[(data1["Implied Volatility BQ"].isna()).sum(),(data1["Time Taken BQ"]).sum(),(data1["Iterations BQ"]).mean()]
    return new_benchmark
benchmark = pd.concat([benchmark, save_benchmark(data, "All classification")])

wanted_choices = ["Slightly ITM", "ATM", "Slightly OTM"]
data=data[data["classification"].isin(wanted_choices)]
benchmark = pd.concat([benchmark, save_benchmark(data, "Excluding Deep ITM/OTM")])
print(benchmark);
string="Expiry Date 14-Aug-2025";
benchmark.to_csv("Benchmark for Time taken, NaN and comparison of IV "+string+".csv");